In [11]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
from faker import Faker

fake = Faker(['en_US'], seed=6090120180)

data = pd.read_csv('supermarket_sales - Sheet1.csv')
data.columns = ['s_' + x.replace(' ', '_').replace('%', '_pct').lower() for x in data.columns]
data.rename(columns={'s_customer_type': 'c_type', 's_gender': 'c_gender'}, inplace=True)
data['c_id'] = data.apply(lambda x: fake.unique.random_int(min=1000, max=9999), axis=1) # generate random customer id
data['s_id'] = data.apply(lambda x: fake.unique.random_int(min=1000, max=9999), axis=1) # generate random sales id
data['s_date'] = data.apply(lambda x: datetime.strptime(x['s_date'], "%m/%d/%Y").strftime('%Y-%m-%d'), axis=1) 

assert (data['c_id'].nunique(), data['s_invoice_id'].nunique(), data['s_id'].nunique()) == (1000, 1000, 1000)

data

,s_invoice_id,s_branch,s_city,c_type,c_gender,s_product_line,s_unit_price,s_quantity,s_tax_5_pct,s_total,s_date,s_time,s_payment,s_cogs,s_gross_margin_percentage,s_gross_income,s_rating,c_id,s_id
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,2019-01-05,13:08,Ewallet,522.83,4.761905,26.1415,9.1,2369,9144
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,2019-03-08,10:29,Cash,76.40,4.761905,3.8200,9.6,8564,1557
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,2019-03-03,13:23,Credit card,324.31,4.761905,16.2155,7.4,2423,4748
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,2019-01-27,20:33,Ewallet,465.76,4.761905,23.2880,8.4,5054,8191
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2019-02-08,10:37,Ewallet,604.17,4.761905,30.2085,5.3,9267,8150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,2019-01-29,13:46,Ewallet,40.35,4.761905,2.0175,6.2,5990,6842
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,2019-03-02,17:16,Ewallet,973.80,4.761905,48.6900,4.4,6960,5675
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2019-02-09,13:22,Cash,31.84,4.761905,1.5920,7.7,1947,6231
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2019-02-22,15:33,Cash,65.82,4.761905,3.2910,4.1,8506,9701


In [12]:

sales_data = data.drop(['c_type', 'c_gender'], axis=1)

# bd data
with open('sales_data.sql', 'w+', encoding='utf-8') as f:
    lines = []
    for i in range(len(sales_data)):
        row = sales_data.iloc[i]
        line = f"INSERT INTO sales (s_id, s_invoice_id, s_branch, s_city, s_product_line, s_unit_price, s_quantity, s_tax_5_pct, s_total, s_date, s_time, s_payment, s_cogs, s_gross_margin_percentage, s_gross_income, s_rating, c_id) VALUES ({row['s_id']}, '{row['s_invoice_id']}', '{row['s_branch']}', '{row['s_city']}', '{row['s_product_line']}', {row['s_unit_price']}, {row['s_quantity']}, {row['s_tax_5_pct']}, {row['s_total']}, '{row['s_date']}', '{row['s_time']}', '{row['s_payment']}', {row['s_cogs']}, {row['s_gross_margin_percentage']}, {row['s_gross_income']}, {row['s_rating']}, {row['c_id']});\n"
        lines.append(line)
        
    f.writelines(lines)

sales_data.to_json('ex3/sales_data.json', orient='records')
sales_data

In [13]:
# sql data

customers_data = data.filter(['c_id', 'c_type', 'c_gender'], axis=1)
customers_data['c_name'] = customers_data.apply(lambda x: fake.first_name_male() + ' ' + fake.last_name() if x['c_gender'] == 'Male' else fake.first_name_female() + ' ' + fake.last_name(), axis=1)
customers_data['c_email'] = customers_data.apply(lambda x: fake.unique.email(), axis=1)
customers_data['c_phone'] = customers_data.apply(lambda x: fake.unique.phone_number(), axis=1)
customers_data['c_birth_date'] = customers_data.apply(lambda x: fake.date_of_birth(minimum_age=18, maximum_age=90), axis=1)

assert (customers_data['c_email'].nunique(), customers_data['c_phone'].nunique()) == (1000, 1000)

with open('customers_data.sql', 'w+', encoding='utf-8') as f:
    f.write('INSERT INTO customers (c_id, c_name, c_email, c_gender, c_phone, c_birth_date, c_type) VALUES\n')
    for i in range(len(customers_data)):
        row = customers_data.iloc[i]
        f.write(f"({row['c_id']}, '{row['c_name']}', '{row['c_email']}', '{row['c_gender']}', '{row['c_phone']}', '{row['c_birth_date']}', '{row['c_type']}')")
        if i == len(customers_data) - 1:
            f.write(';')
        else:
            f.write(',\n')
        
customers_data.to_json('customers_data.json', orient='records')
customers_data

,c_id,c_type,c_gender,c_name,c_email,c_phone,c_birth_date
0,2369,Member,Female,Sandra Moody,davidchapman@example.com,+1-400-792-2006x6423,1992-03-21
1,8564,Normal,Female,Kathy Gardner,kennedyeric@example.com,7318492591,1939-10-25
2,2423,Normal,Male,John Levy,ileonard@example.net,001-240-489-2941x087,1985-04-13
3,5054,Member,Male,Jeffrey Fitzgerald,oscar48@example.com,+1-967-138-0634x269,1937-03-13
4,9267,Normal,Male,Jorge White,pamelaperez@example.com,+1-359-792-3848x8366,1992-09-18
...,...,...,...,...,...,...,...
995,5990,Normal,Male,Todd Owens,wwilliams@example.org,+1-459-777-2758x01973,1936-09-08
996,6960,Normal,Female,Sarah Perez,youngjennifer@example.org,(499)947-5279x721,1993-12-06
997,1947,Member,Male,Alexander Weber,alyssa87@example.org,976.817.2591,1976-09-16
998,8506,Normal,Male,Dustin Hall,kwhite@example.org,024-028-8497x6863,1971-05-31


In [14]:
# MOON queries

queries = [
    # Q1: name of customers who bought the top 10 most expensive products
    "select c_name, s_unit_price from customers c join sales s on c.c_id = s.c_id order by s.s_unit_price desc limit 10;",
    # Q2: total number of product sales by customer type
    "select c_type, sum(s_total) from customers c join sales s on c.c_id = s.c_id group by c_type;",
    # Q3: top 3 most profitable months in sales made to females
    "select extract(month from s_date) as s_month, sum(s_unit_profit) as s_profit_sum from customers c, (select c_id, (s_unit_price - s_tax_5_pct) as s_unit_profit, s_date from sales) s where c.c_id = s.c_id and c.c_gender = 'Female' group by s_month order by s_profit_sum desc limit 3;",
]
for n, query in enumerate(queries, 1):
    with open(f'q{n}.sql', 'w+', encoding='utf-8') as f:
        f.write(query)

In [ ]:
# NoSQL queries
nosql_queries = [
    # Q1: name of customers who bought the top 10 most expensive products
    'db.assets.find({}).sort({s_unit_price: -1}).limit(10)'
    # Q2: total number of product sales by customer type
    # Q3: top 3 most profitable months in sales made to females
]
for n, query in enumerate(nosql_queries, 1):
    with open(f'q{n}.mongoq', 'w+', encoding='utf-8') as f:
        f.write(query)

In [15]:
# BigDAWG queries
cast = "bdcast(bdtext({'op': 'scan', 'table': 'sales'}), sales_acc_%i, '(s_invoice_id varchar, s_branch varchar, s_city varchar, s_product_line varchar, s_unit_price decimal, s_quantity int, s_tax_5_pct decimal, s_total decimal, s_date date, s_time varchar, s_payment varchar, s_cogs decimal, s_gross_margin_percentage decimal, s_gross_income decimal, s_rating decimal, c_id bigint)', relational)"

host = "192.168.0.104:8080"

for n, query in enumerate(queries, 1):
    with open(f'bdg_q{n}.sh', 'w+', encoding='utf-8') as f:
        f.write('curl -X POST -d "bdrel(' + query.replace("sales", f"{cast % n}") + f')" http://{host}/bigdawg/query')